In [2]:
# Command to make `src` module available in notebook
# Details: https://stackoverflow.com/a/35273613

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


### Поиск аномалий 

В этом файле копирования файлов и подключения устройств будут считаться подозрительными при условии, что им предшествует подозриттельный вход

In [3]:
from src.cert import CERTDataType
from src.data_factory import get_data_stream
from src.indicators.meta_logon import MetaLogonIndicator


Загружаем события:

In [4]:
events = get_data_stream([
    CERTDataType.device,
    CERTDataType.file,
    CERTDataType.logon,
])


Инициализируем индикатор с настройками
- Вход считается подозрительным, если он первый и произошел ранее, чем за час до среднего времени первого входа. Аналогично выход считается подозрительным, если произошел более чем через час после среднего времени последнего выхода
- Копирование файлов или подключение устройств считается подозрительным, если за последние два часа это событие произошло больше раз, чем происходило в первые 90 дней

Далее выбираем событич, на которых сработал индикатор

In [7]:
indicator = MetaLogonIndicator(workday_delta=60, usage_interval=120)
bad_events = []

for event in events:
    val = indicator.process(event)
    if val:
        bad_events.append(event)


In [8]:
set(x.user_id for x in bad_events)


{'RCW0822'}

Пользователь `RCW0822` – инсайдер, который совершал атаку по сценарию 1 (резко возросло количество входов в нерабочее время и исползование внешних устройств). Инсайдер найден верно

In [10]:
[x for x in bad_events]


[DeviceUsageEvent(event_id='{J4L9-S1FN89GY-5468VAXS}', user_id='RCW0822', timestamp=datetime.datetime(2010, 10, 2, 5, 54, 26)),
 FileCopyEvent(event_id='{U7P1-Q5KH75DG-4706FRLP}', user_id='RCW0822', timestamp=datetime.datetime(2010, 10, 2, 6, 14, 59)),
 DeviceUsageEvent(event_id='{C2K9-H6XD05WT-3372YRPP}', user_id='RCW0822', timestamp=datetime.datetime(2010, 10, 2, 6, 29, 20)),
 DeviceUsageEvent(event_id='{K1J2-B1QY57TD-5843NFPH}', user_id='RCW0822', timestamp=datetime.datetime(2010, 10, 8, 6, 55, 47)),
 FileCopyEvent(event_id='{N9S5-U2EQ65CP-9474SERZ}', user_id='RCW0822', timestamp=datetime.datetime(2010, 10, 8, 6, 57, 35)),
 DeviceUsageEvent(event_id='{T8L8-S2GW60IM-6106YRLE}', user_id='RCW0822', timestamp=datetime.datetime(2010, 10, 8, 6, 58, 5)),
 DeviceUsageEvent(event_id='{L4P5-O1DB27TE-3676CNNU}', user_id='RCW0822', timestamp=datetime.datetime(2010, 10, 15, 5, 2, 40)),
 FileCopyEvent(event_id='{W7T9-A1QI23BP-8302WBIF}', user_id='RCW0822', timestamp=datetime.datetime(2010, 10, 15

События выше являются частю инсайдерской атаки. При этом всего событий в атаке 24. Если засчитат входы, предшествующие подозрителным действиям из `bad_events`, было обнаружено 11 подозрительных активностей из 24, то есть точность составляет 1.0, а полнота 0.45